<h1>CHORDIFY: </h1>
This project connects to Hooktheory's Chord Progression API and Spotify's API to provide users with an easy way to discover songs that feature their chosen chord progressions.Workflow:
<ul>
    <li><strong>User Input:</strong> The user is prompted to enter a chord progression, the desired number of pages(of results) and the instrument they want to play this progression on.</li>
    <li><strong>Fetch Songs:</strong> The Interface calls the Hooktheory API using the specified chord progression and my Authorisation key and recieves songs</li>
    <li><strong>Process Results:</strong> The fetched songs are processed to create a structured dataframe, including artist names, song titles and where the chord progression appears in the song.</li>
    <li><strong>Spotify Lookup:</strong> The application uses the Spotify API to create a playlist of the fetched songs.</li>
    <li><strong>Youtube Playlist:</strong> Finally, the application  looks for Youtube tutorials for each of these songs for the instrument that they specified in the user input section</li>
</ul>


In [27]:
import os # For working with files and folders
import pickle # Used to save and load data 
import requests # Makes it easier to send HTTP requests
import json # To work with JSON data 
import pandas as pd # Library for working with data - for the dataframe
import random # To make random numbers
import collections # Gives extra tools for storing data in different ways
import spotipy # For using Spotify with Python
import spotipy.util as util
import webbrowser # Opens links in the web browser
import urllib.request # For sending requests 
import googleapiclient # Main Google API library
from googleapiclient.discovery import build  # To set up the Google API service
import google_auth_oauthlib.flow # Helps with signing in to Google
import googleapiclient.errors # For Google API error messages
from google.auth.transport.requests import Request  # To send secure requests
from google_auth_oauthlib.flow import InstalledAppFlow  # For Google login flow

<h2>Retrieving my Activation Key for the HookTheory API</h2>
I created a json file with my username, password and active key and then created a function which uses the key to acess Hooktheories database. Authentication code on Hooktheory wesbsite:/www.hooktheory.com/api/trends/docs#child-path was used to help with my code

In [28]:
BASE_URL = "https://api.hooktheory.com/v1/"
# Read my Deatils.Json file to get my active key 
with open('Details.json', 'r') as file:
    credentials = json.load(file)
    auth_token = credentials['active_key']  

# Use the function ref:(https://www.hooktheory.com/api/trends/docs#child-path) to find songs with a certain chord_path
def get_songs_with_progression(auth_token, chord_path, page=1): # a Function that recives my authenitcaion code, the chord progression of choice and the pages to fecth amount
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {auth_token}"
    }
    url = BASE_URL + f"trends/songs?cp={chord_path}&page={page}"
    response = requests.get(url, headers=headers)
    
    # Check for successful response and return songs, or empty list on failure
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data from API: {response.status_code} - {response.text}")
        return []

<h2>User Input: Please enter the chord progression you’re interested in:</h2>

A chord progression is a sequence of chords played in a specific order that forms the harmonic foundation of a song. Here are examples:
<ol>
    <li><strong>Pop/Rock:</strong> 1, 5, 6, 4 (C, G, Am, F)</li>
    <li><strong>Jazz:</strong> 2, 5, 1 (Dm7, G7, Cmaj7)</li>
    <li><strong>Blues:</strong> 1, 4, 5 (C, F, G)</li>
    <li><strong>Country:</strong> 1, 4, 5 (C, F, G)</li>
    <li><strong>R&B/Soul:</strong> 1, 6, 4, 5 (C, Am, F, G)</li>
    <li><strong>Folk:</strong> 1, 4, 5 (C, F, G)</li>
    <li><strong>Classical:</strong> 1, 4, 5, 1 (C, F, G, C)</li>
    <li><strong>Reggae:</strong> 1, 4, 5 (C, F, G)</li>
</ol>


<h3>Notes on Formatting:</h3>
<ul>
    <li> Your chord input needs to be presented as numbers and separated by commas (matching the number format above)</li>
</ul>

In [38]:
# Function to get user input for chord progression
def get_user_input():
    user_input = input("Please enter a chord progression (e.g., '4,1'): ")
    print("You entered:", user_input)
    return user_input

# Function to get the number of pages to fetch
def get_number_of_pages():
    num_pages = int(input("Please enter the number of pages to fetch (e.g., '5'- note: each page can collect up to 0 songs): "))
    return num_pages
    
#Function to retrieve the users instrument of choice
def instrument_of_choice():
    instrument = str(input("Please enter the instrument which you are interested in playing this chord progression on (e.g. piano, guitar etc)"))
    return instrument

# Call the function to prompt for input
user_input = get_user_input()
num_pages = get_number_of_pages()
instrument = instrument_of_choice()

Please enter a chord progression (e.g., '4,1'):  1,4,1


You entered: 1,4,1


Please enter the number of pages to fetch (e.g., '5'- note: each page can collect up to 0 songs):  1
Please enter the instrument which you are interested in playing this chord progression on (e.g. piano, guitar etc) Bass Guitar


In [39]:
#Store instrument of choiceand chord progression in a new variable
choice_of_instrument = instrument
chord_progress= user_input

<h2> Create a Song List Data Frame </h2>

In [40]:
# Print the Title of What the Data Frame represents
print(f"\nSongs with {user_input} progression:")

# Initialise an empty list to hold all songs found
all_songs = []

# Fetch songs from multiple pages
for page in range(1, num_pages + 1):
    songs = get_songs_with_progression(auth_token, user_input, page)
    if not songs:  # Stop if there are no songs found 
        print(f"No more songs found on page {page}. Stopping fetch.")
        break
    all_songs.extend(songs)  # append the earlier initialised all_songs list 


song_data = []
# For each song found from the API, create a dictionary containing the artist's name and the song title
for song in all_songs:
    song_data.append({
        'Artist': song['artist'],
        'Song': song['song'],
        'Section':song['section']
    })

# Create a DataFrame from the song_data list
song_df = pd.DataFrame(song_data)

# Clean the columns to ensure consistent grouping
song_df['Artist'] = song_df['Artist'].str.strip().str.title()  
song_df['Song'] = song_df['Song'].str.strip().str.title()     

# Group by Artist and Song, and combine the Section column #Aggrigate function to combine the section column if the song has been repeated
combined_df = song_df.groupby(['Artist', 'Song'], as_index=False).agg({'Section': ', '.join}) 

#Find the length of songs generated:
number_songs = len(combined_df)
print(number_songs)

# Print the resulting DataFrame
print(combined_df)



Songs with 1,4,1 progression:
16
                           Artist                             Song  \
0                       Aerosmith                           Cryin'   
1                 Black Eyed Peas            Just Can'T Get Enough   
2                     Buddy Holly               That'Ll Be The Day   
3                        Daughtry                             Home   
4             Death Cab For Cutie  I Will Follow You Into The Dark   
5                    Dire Straits                        Skateaway   
6                      Don Mclean                     American Pie   
7                      Elton John                            Levon   
8                             Fun                      Some Nights   
9                    Jack Johnson                            Flake   
10  James Rich And Boots Randolph                       Yakety Sax   
11                    John Lennon                          Imagine   
12                    Johnny Cash                  I Wal

<h1>Retrieving my Spotify API information </h1>

Reference: https://github.com/katsully/intro-creative-code-year3

In [41]:
# Save the contents of the spotify Json into a new file called credentials 
credentials = "spotify-keys.json"
# With not only opens a file but closes it once the contents of the read section is complete
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [14]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [15]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [16]:
sp = spotipy.Spotify(auth=token)

<h2> Creating a Spotify Playlist for users with songs within their chord Progression of Interest</h2>

In [42]:
# Create an empty list to store song URIs
songs = []

# Loop through each row in the data frame of songs with the users preffered chord progression 
for index, row in combined_df.iterrows():
    artist = row['Artist']
    title = row['Song']
    # Create the query string to search by artist and title
    query = f"artist:{artist} track:{title}"
    # Search for the track in Spotify
    search_results = sp.search(q=query, type="track")
    # Check if any results were returned
    if len(search_results['tracks']['items']) > 0:
        # Get the URI of the first search result
        first_song = search_results['tracks']['items'][0]
        songs.append(first_song['uri'])
    else:
        # Optionally, handle the case where no results are found
        print(f"No match found for '{title}' by '{artist}'.")
print(songs)
output_of_songs= len(songs)

No match found for 'Yakety Sax' by 'James Rich And Boots Randolph'.
['spotify:track:4t1fWWJQs5V9YErfsrDslC', 'spotify:track:3JA9Jsuxr4xgHXEawAdCp4', 'spotify:track:4UcHTV3TjlThmMlZgOG4Kr', 'spotify:track:5vjbBPVd4ncMex8zf2V10o', 'spotify:track:2ndWbjiiNBEOrlfToKlABE', 'spotify:track:22KSOpUAlJLJhACvQ65Bho', 'spotify:track:1fDsrQ23eTAVFElUMaf38X', 'spotify:track:0aJfFLl0grcQS7euiIp0ni', 'spotify:track:67WTwafOMgegV6ABnBQxcE', 'spotify:track:0yo8lQMdjioEBW7eux4X0L', 'spotify:track:7pKfPomDEeI4TPT6EOYjn9', 'spotify:track:7hxZF4jETnE5Q75rKQnMjE', 'spotify:track:6ueRLh9GIV8DccBixZ1lFT', 'spotify:track:5KqldkCunQ2rWxruMEtGh0', 'spotify:track:5YaGvzrA6nlTElsJwT6NcZ']


In [21]:
my_playlist = sp.user_playlist_create(user=username, name=f"Songs with a {user_input} chord progression", public=True,
                                      description=f"A playlist of {output_of_songs} songs with the {user_input} chord progression")
results = sp.user_playlist_add_tracks(username, my_playlist['id'], songs)

print(results)

{'snapshot_id': 'AAAAAm9aIXqxgZObJqqWT2+SsM4NAUGR'}


In [19]:
webbrowser.open(my_playlist['external_urls']['spotify']) # Open up the Playlist in my Web Player

True

<h2> Create a Youtube Playlists to Help Users Learn these songs on their instrument of choice </h2>
Code found from yotube Tutorial:https://www.youtube.com/watch?v=vQQEaSnQ_bs

In [43]:
# Scopes for YouTube API
SCOPES = [
    'https://www.googleapis.com/auth/youtube.force-ssl'  # Scope needed for creating playlists
]
credentials = None 
# token.pickle stores the user's credentials from previously successful logins
if os.path.exists('token.pickle'):
    print('Loading Credentials From File...')
    with open('token.pickle', 'rb') as token:
        credentials = pickle.load(token)

# If there are no valid credentials available, then either refresh the token or log in.
if not credentials or not credentials.valid:
    if credentials and credentials.expired and credentials.refresh_token:
        print('Refreshing Access Token...')
        credentials.refresh(Request())
    else:
        print('Fetching New Tokens...')
        flow = InstalledAppFlow.from_client_secrets_file(
            'client_secret.json',
            scopes=SCOPES
        )

        flow.run_local_server(port=8080, prompt='consent', authorization_prompt_message='')
        credentials = flow.credentials

        # Save the credentials for the next run
        with open('token.pickle', 'wb') as f:
            print('Saving Credentials for Future Use...')
            pickle.dump(credentials, f)

# Build the YouTube service
youtube = build("youtube", "v3", credentials=credentials)

Loading Credentials From File...
Refreshing Access Token...


<h2>Configuring my Youtube API Account </h2>
Using Documentation: https://developers.google.com/youtube/v3/guides/implementation/playlists & https://stackoverflow.com/questions/72029929/create-playlist-by-youtube-data-api-python

In [44]:
def create_playlist(title, description):
    request = youtube.playlists().insert(
        part="snippet,status",
        body={
            "snippet": {
                "title": title,
                "description": description
            },
            "status": {
                "privacyStatus": "public"  # Make the playlist PUBLIC
            }
        }
    )
    response = request.execute()
    return response['id']  # Return the ID of the created playlist

In [ ]:
def add_tutorials_to_playlist(playlist_id):
# Define a function that iterates through the data frame and creates a query for YouTube search
    for index, row in combined_df.iterrows(): # For aech row within the data frame get the artist and the title 
        artist = row['Artist']
        title = row['Song']
        instrument = choice_of_instrument 
        query = f"{title} by {artist} - {instrument} tutorial" # a query of the tutorials wanted
        # Search for the video
        try:
            search_request = youtube.search().list(
                q=query,
                part="snippet",
                type="video",
                maxResults=1
            )
            search_response = search_request.execute()
            # Select the first video reuslts from the search 
            if search_response['items']:
                video_id = search_response['items'][0]['id']['videoId']
                print(f"Adding video ID: {video_id} to playlist: {playlist_id}")
                
                # Add the video to the playlist
                playlist_item_request = youtube.playlistItems().insert(
                    part="snippet",
                    body={
                        "snippet": {
                            "playlistId": playlist_id, 
                            "resourceId": {
                                "kind": "youtube#video",
                                "videoId": video_id
                            }
                        }
                    }
                )
                playlist_item_request.execute()
                print(f"Video ID: {video_id} added successfully.")
            else:
                print(f"No results found for: {query}")
        except Exception as e:
            print(f"An error occurred while processing '{query}': {e}")

# Playlist creation logic
playlist_title = f"{chord_progress} chord progression songs - {instrument} tutorials"
playlist_description = f"A collection of tutorials for songs with a {chord_progress} chord progression. Play along with your Spotify Playlist when you become a Pro"
playlist_id = create_playlist(playlist_title, playlist_description)
add_tutorials_to_playlist(playlist_id)

print(f"Playlist of {instrument} tutorials successfully created!") # The Playlist: https://www.youtube.com/watch?v=3_wxvt4P3IE&list=PLm17GLd3q9-bvn0n1-EU0x3MH1ZKslJgM